In [ ]:
#!pip install geemap
#!pip install ndvi2gif
#!pip install geedim

In [1]:
import geemap
import ee
from ndvi2gif import NdviSeasonality

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

We utilize the map drawing tools to create a roi

In [ ]:
roi = Map.draw_last_feature

In [ ]:
s = NdviSeasonality(roi="s2:29SQA", start_year=2022, end_year=2022, sat="Landsat")
print(s.roi.getInfo())

In [3]:
# Here we select ndwi max for 12 periods seasons in Andalucia
myclass = NdviSeasonality(roi="s2:29SQA", sat='S2', key='median', periods=4,start_year=2016, end_year=2024, index='ndwi')
s2 = myclass.get_year_composite()

vizParams = {'bands': ['winter', 'spring', 'summer'], 'min': 0, 'max':0.5}

Map.addLayer(s2, vizParams, 'P90')

There we go again...
Loading Sentinel-2 MGRS tile from GitHub...
Found Sentinel-2 tile for 29SQA


In [4]:
# ✅ NUEVAS POSIBILIDADES:
# 6 períodos (~2 meses cada uno)
myclass_6 = NdviSeasonality(roi="s2:29SQA", sat='S2', key='median', periods=6, 
                            start_year=2016, end_year=2024, index='ndvi')
s2_6 = myclass_6.get_year_composite()
# Bandas disponibles: ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

# 8 períodos (~1.5 meses cada uno)  
myclass_8 = NdviSeasonality(roi="s2:29SQA", sat='S2', key='median', periods=8,
                            start_year=2016, end_year=2024, index='ndvi')
s2_8 = myclass_8.get_year_composite()
# Bandas disponibles: ['p1', 'p2', ..., 'p8']

# Visualizar los primeros 3 períodos de cualquier configuración
vizParams = {'bands': myclass_6.period_names[:3], 'min': 0, 'max': 0.5}
Map.addLayer(s2_6, vizParams, 'NDvI 6 periods')

There we go again...
Loading Sentinel-2 MGRS tile from GitHub...
Found Sentinel-2 tile for 29SQA
There we go again...
Loading Sentinel-2 MGRS tile from GitHub...
Found Sentinel-2 tile for 29SQA


In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

def convert_3D_2D(geometry):
    new_geo = []
    for geom in geometry:
        if hasattr(geom, "has_z") and geom.has_z:
            if geom.geom_type == 'Polygon':
                exterior = [xy[:2] for xy in geom.exterior.coords]
                interiors = [[xy[:2] for xy in ring.coords] for ring in geom.interiors]
                new_geom = Polygon(exterior, interiors)
                new_geo.append(new_geom)
            elif geom.geom_type == 'MultiPolygon':
                parts = []
                for part in geom.geoms:
                    exterior = [xy[:2] for xy in part.exterior.coords]
                    interiors = [[xy[:2] for xy in ring.coords] for ring in part.interiors]
                    parts.append(Polygon(exterior, interiors))
                new_geo.append(MultiPolygon(parts))
        else:
            new_geo.append(geom)  # ya es 2D o no tiene .has_z
    return new_geo

# Cargar el archivo
gdf = gpd.read_file("../data/s2tiles.geojson")

# Convertir geometrías a 2D si es necesario
gdf["geometry"] = convert_3D_2D(gdf["geometry"])

# Verificar resultado
print(gdf.geometry.iloc[0].wkt[:200])  # Muestra el principio del WKT para confirmar que no hay Z

# Guardar el nuevo archivo
gdf.to_file("../data/s2tiles_clean.geojson", driver="GeoJSON")

In [ ]:
# Doñana NDVI 
# Here we select ndwi max for 12 periods seasons
# Show graphics and pixel values
# Also change to 24 periods

myclass = NdviSeasonality(roi=roi, sat='S2', key='max', periods=12,start_year=1984, end_year=2024, index='ndmi')
s2 = myclass.get_year_composite()

vizParams = {'bands': ['january', 'november', 'july'], 'min': 0, 'max':0.8}

Map.addLayer(s2, vizParams, 'P90')

Of course, it's possible to use loops to load several data at once

In [ ]:
# Here we download a gif for all the years in our class with the selected bands
myclass.get_gif(bands=['january', 'november', 'july'])

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = Map.draw_last_feature

In [ ]:
# Statistics loop

stats = ['max', 'median', 'mean', 'perc_90', 'perc_95']
vizParams = {'bands': ['march', 'april', 'june'], 'min': 0, 'max':0.8}

for i in stats:

    myclass = NdviSeasonality(roi=roi, sat='S2', key=i, periods=12,start_year=2020, end_year=2022, index='ndvi')
    s2 = myclass.get_year_composite()
    Map.addLayer(s2, vizParams, str(i))

In [ ]:
# Years loop
# Change roi to Doñana deimsID

years = [i for i in range(2018, 2023)]

vizParams = {'bands': ['march', 'april', 'june'], 'min': 0, 'max':0.8}

for y in years:
    
    name = 'maxNDVI_' + str(y)
    myclass = NdviSeasonality(roi=roi, sat='S2', key='perc_90', periods=12,start_year=y, end_year=y+1, index='ndvi')
    s2 = myclass.get_year_composite()
    Map.addLayer(s2, vizParams, name)

We could do something bigger. 
Africa Example with MODIS

In [ ]:
Map = geemap.Map()
Map

In [ ]:
import ee
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq("wld_rgn", "Africa"))

In [ ]:
# Africa NDVI Mean
# Also show Nile delta with S2, take a sight on what you can see over the desert

Africa = NdviSeasonality(roi=roi, sat='MODIS', key='mean', periods=4,start_year=2020, end_year=2022, index='ndvi')
MODIS = Africa.get_year_composite()

vizParams = {'bands': ['summer', 'winter', 'spring'], 'min': 0.1, 'max':0.8}
Map.addLayer(MODIS, vizParams, 'S2_P95_24')

We said deimsPY allows to select your eLTER site qith only using your deimsID, so let's check it out

In [ ]:
Map = geemap.Map()
Map

In [ ]:
!pip install deims

In [ ]:
# Braila Island Flood

myclass = NdviSeasonality(roi='deimsid:https://deims.org/d4854af8-9d9f-42a2-af96-f1ed9cb25712', sat='S2', key='perc_95', periods=12,start_year=2017, end_year=2022, index='ndwi')
s2 = myclass.get_year_composite()
vizParams = {'bands': ['october', 'june', 'january'], 'min': 0.1, 'max':0.8}
Map.addLayer(s2, vizParams, 'Braila Island P95')

Let's try to get the maximum flood in al januarys

In [ ]:
myclass.get_export()

In [ ]:
# Max january flood

januaryFloodmax = s2.select('january').max()
Map.addLayer(januaryFloodmax, {'min': 0, 'max':0.8}, 'January Max Flood P90')

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = Map.draw_last_feature

In [ ]:
# Doñana NDVI Stats
myclass = NdviSeasonality(roi='deimsid:https://deims.org/bcbc866c-3f4f-47a8-bbbc-0a93df6de7b2', sat='Landsat', 
                          key='max', periods=12,start_year=2019, end_year=2020, index='ndvi')
Landsat = myclass.get_year_composite()
vizParams = {'bands': ['january', 'april', 'september'], 'min': 0.1, 'max':0.8}

Map.addLayer(Landsat, vizParams, 'Max')

We can also have zonal statistic

In [ ]:
# Points and polygons zonal stats

import os

# This is a Google Earth Engine Feature Collection
alcornoques = ee.FeatureCollection('users/digdgeografo/pajarera')

# This is a local point shapefile
samples = '/home/diego/samples.shp'
geom = geemap.shp_to_ee(samples)

# Define the ouptut shape
out_shp = os.path.join(os.getcwd(), 'mystatstets_oaks.shp')

# Compute the statistics
geemap.zonal_statistics(Landsat, in_zone_vector=alcornoques, out_file_path=out_shp, statistics_type='MEAN', scale=30)

It is also possible to export the collections. There's a problem qith size limits from GEE but small areas are ok

In [ ]:
# Take a smaller roi

roi = Map.draw_last_feature

In [ ]:
# Here we select ndwi max for 12 periods seasons in Andalucia

myclass = NdviSeasonality(roi=roi, sat='Landsat', key='max', periods=12,start_year=2017, end_year=2020, index='ndvi')
Landsat = myclass.get_year_composite()
vizParams = {'bands': ['january', 'april', 'september'], 'min': 0.1, 'max':0.8}

Map.addLayer(Landsat, vizParams, 'Max')

In [ ]:
# Export it and load the data in QGIS

myclass.get_export()

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = Map.draw_last_feature

In [ ]:
# Last but not least and exaample of Sierra Nevada NDSI

myclass = NdviSeasonality(roi=roi, sat='S2', key='median',
                          periods=12,start_year=2020, end_year=2022, index='ndsi')
s2 = myclass.get_year_composite()

vizParams = {'bands': ['january', 'february', 'march'], 'min': 0, 'max':0.8}
Map.addLayer(s2, vizParams, 'S2_MAX_Sierra Nevada')